In [1]:
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt

# 2-bus, 1-readout

In [59]:
def calculate_qubit_2bus_1read_equiv_cap(C_M, index):
    """
    Calculate effective shunt and coupling capacitance.
    Arg:
        C_M : Maxewll capacitance matrix
        index : index of each conductor in C_M, starting from 0
    Return:
        Tuple of effective bus, readout and shunt capacitance
    """
    index_bus1 = index['bus1']
    index_bus2 = index['bus2']
    index_gnd = index['gnd']
    index_island1 = index['island1']
    index_island2 = index['island2']
    index_read = index['read']
    
    NUM_CONDUCTOR = C_M.shape[0]
    
    # find beta_bus1, beta_bus2 and beta_readout
    #, where beta_bus1 = C_bus1/C_sigma 
    # and C_sigma = C_bus1 + C_bus2 + C_read + C_shunt

    C_M_1 = np.zeros((NUM_CONDUCTOR, NUM_CONDUCTOR))
    C_M_1[:, index_island1] = C_M[:, index_island1]
    C_M_1[:, index_island2] = C_M[:, index_island2]
    C_M_1[index_bus1, index_bus1] = -1
    C_M_1[index_bus2, index_bus2] = -1
    C_M_1[index_gnd, index_gnd] = -1
    C_M_1[index_read, index_read] = -1

    b1 = np.linalg.solve(C_M_1, -C_M[:, index_bus1])
    beta_bus1 = b1[index_island1] - b1[index_island2]

    b2 = np.linalg.solve(C_M_1, -C_M[:, index_bus2])
    beta_bus2 = b2[index_island1] - b2[index_island2]
    
    r = np.linalg.solve(C_M_1, -C_M[:, index_read])
    beta_read = r[index_island1] - r[index_island2]
    
    # Find C_sigma
    # V_island1 - V_island2 = 1, Eq.1
    # Q_island1 + Q_island2 = 0, Eq.2
    # From Eq1 and Eq2, find V_island1 and V_island2 

    A = np.array([[1, -1],
                 [C_M[index_island1, index_island1]+C_M[index_island2, index_island1]
                  , C_M[index_island1, index_island2]+C_M[index_island2, index_island2]]])

    b = np.array([[1]
                  ,[0]])
    V = np.linalg.solve(A, b)  # A*V = b, solve for V[0]=V_island1, V[1]=V_island2

    V0 = np.zeros(NUM_CONDUCTOR)
    V0[index_island1] = V[0]
    V0[index_island2] = V[1]

    Q = C_M.dot(V0)
    C_sigma = np.dot(Q, V0)
      
    # find C_bus1, C_bus2, C_read and C_shunt
    A = np.array([[beta_bus1-1, beta_bus1, beta_bus1, beta_bus1]
                  ,[beta_bus2, beta_bus2-1, beta_bus2, beta_bus2]
                ,[beta_read, beta_read, beta_read-1, beta_read]  
                ,[1, 1, 1, 1]])
    b = np.array([[0]
                  ,[0]
                  ,[0]
                 ,[C_sigma]])
    c = np.linalg.solve(A, b)
    
    return c

def print_result_2bus_1read(c):
    c = np.ndarray.flatten(c)
    print(f'C_bus1 = {c[0]:.3f} fF\n' 
         + f'C_bus2 = {c[1]:.3f} fF\n' 
         + f'C_read = {c[2]:.3f} fF\n'
         + f'C_shunt = {c[3]:.3f} fF')
    

In [60]:
C_M = np.array([[107.94,   -0.072728, -96.56, -0.72528, -9.7239,-0.041219]
               ,[-0.072728, 107.72, -95.966, -0.72699, -9.5856, -0.041242]
               ,[-96.56,   -95.966, 586.48,  -59.967, -55.133, -95.917]
               ,[-0.72528, -0.72699, -59.967, 92.027,  -23.543,   -7.0491]
               ,[-9.7239, -9.5856,-55.133, -23.543, 98.962,-0.61872]
               ,[-0.041219,-0.041242, -95.917,   -7.0491, -0.61872, 103.03]
              ]) 

# index dictionary of conductors in input Maxwell capacitance matrix
conductor_index = { 'gnd' : 2
                   , 'island1' : 3
                   , 'island2' : 4
                   , 'bus1' : 0
                   , 'bus2' :1
                   , 'read' : 5
                  }

In [61]:
result_2bus_1read = calculate_qubit_2bus_1read_equiv_cap(C_M, conductor_index)
print_result_2bus_1read(result_2bus_1read)

C_bus1 = -4.248 fF
C_bus2 = -4.181 fF
C_read = 3.400 fF
C_shunt = 64.464 fF


# 1-bus, 1-readout

In [4]:
def update_mutual_capacitance(Maxwell_cap_matrix,conductor1_index, conductor2_index, new_cap):
    """
    Args:
        Maxwell_cap_matrix: Maxwell capacitance matrix from Q3D
        conductor1_index : index of conductor 1
        conductor2_index : index of conductor 2
        new_cap : new capacitance for replacement
    """
    
    if conductor1_index == conductor2_index:
        raise('Error. First and second conductor index must be different!')
    
    M = np.copy(Maxwell_cap_matrix)
    
    M[conductor1_index, conductor2_index] = -new_cap
    M[conductor2_index, conductor1_index] = -new_cap
    
    for i in range(M.shape[0]):
        M[i,i] = -(np.sum(M[i,:]) - M[i,i])
        
    return M

def calculate_qubit_1bus_1read_equiv_cap(C_M, index):
    """
    Calculate effective shunt and coupling capacitance.
    Arg:
        C_M : Maxewll capacitance matrix
        index : index of each conductor in C_M, starting from 0
    Return:
        Tuple of effective bus, readout and shunt capacitance
    """
    index_bus1 = index['bus1']
    index_gnd = index['gnd']
    index_island1 = index['island1']
    index_island2 = index['island2']
    index_read = index['read']
    
    NUM_CONDUCTOR = C_M.shape[0]
    
    C_M_1 = np.zeros((NUM_CONDUCTOR, NUM_CONDUCTOR))
    C_M_1[:, index_island1] = C_M[:, index_island1]
    C_M_1[:, index_island2] = C_M[:, index_island2]
    C_M_1[index_bus1, index_bus1] = -1
    C_M_1[index_gnd, index_gnd] = -1
    C_M_1[index_read, index_read] = -1

    b1 = np.linalg.solve(C_M_1, -C_M[:, index_bus1])
    beta_bus1 = b1[index_island1] - b1[index_island2]

    r = np.linalg.solve(C_M_1, -C_M[:, index_read])
    beta_read = r[index_island1] - r[index_island2]
    
    # Find C_sigma
    # V_island1 - V_island2 = 1, Eq.1
    # Q_island1 + Q_island2 = 0, Eq.2
    # From Eq1 and Eq2, find V_island1 and V_island2 

    A = np.array([[1, -1],
                 [C_M[index_island1, index_island1]+C_M[index_island2, index_island1]
                  , C_M[index_island1, index_island2]+C_M[index_island2, index_island2]]])

    b = np.array([[1]
                  ,[0]])
    V = np.linalg.solve(A, b)  # A*V = b, solve for V[0]=V_island1, V[1]=V_island2

    V0 = np.zeros(NUM_CONDUCTOR)
    V0[index_island1] = V[0]
    V0[index_island2] = V[1]
    V0[index_bus1] = 0
    V0[index_read] = 0
    V0[index_gnd] = 0

    Q = C_M.dot(V0)
    C_sigma = np.dot(Q, V0)
    
    # find C_bus1, C_read and C_shunt
    A = np.array([[beta_bus1-1, beta_bus1, beta_bus1]
                ,[beta_read,  beta_read-1, beta_read]  
                ,[1, 1, 1]])
    b = np.array([[0]
                  ,[0]
                 ,[C_sigma]])
    c = np.linalg.solve(A, b)
    
    return c

def print_result_1bus_1read(c):
    c = np.ndarray.flatten(c)
    print(f'C_bus1 = {c[0]:.3f} fF\n' 
         + f'C_read = {c[1]:.3f} fF\n'
         + f'C_shunt = {c[2]:.3f} fF')
    

In [5]:
C_M = np.array([[107.94, -96.56, -0.72528, -9.7239,-0.041219]
               ,[-96.56, 586.48,  -59.967, -55.133, -95.917]
               ,[-0.72528, -59.967, 92.027,  -23.543,   -7.0491]
               ,[-9.7239, -55.133, -23.543, 98.962,-0.61872]
               ,[-0.041219, -95.917,   -7.0491, -0.61872, 103.03]
              ])

# index dictionary of conductors in input Maxwell capacitance matrix
conductor_index = { 'gnd' : 1
                   , 'island1' : 2
                   , 'island2' : 3
                   , 'bus1' : 0
                   , 'read' : 4
                  }

In [6]:
cap_result = calculate_qubit_1bus_1read_equiv_cap(C_M, conductor_index)
print_result(cap_result)

C_bus1 = -4.248 fF
C_read = 3.400 fF
C_shunt = 60.283 fF


In [7]:
C_M_1 = update_mutual_capacitance(C_M, 0, 3, 50)
cap_result1 = calculate_qubit_1bus_1read_equiv_cap(C_M_1, conductor_index)
print_result(cap_result1)

C_bus1 = -19.081 fF
C_read = 4.055 fF
C_shunt = 79.860 fF


In [8]:
7e-15/2*np.sqrt(6.26e9*5.05e9/55e-15/468e-15)/1e6

122.6582117411505

# ZZ gate, qubit 

In [9]:
C_M = np.array([[143.07500,-87.36528,-2.23553,-52.84648,-0.23655]
               ,[-87.36528,294.22820,-40.74111,-46.17334,-77.96845]
               ,[-2.23553,-40.74111,67.58779,-15.71175,-7.76007]
               ,[-52.84648,-46.17334,-15.71175,118.06270,-0.89528]
               ,[-0.23655,-77.96845,-7.76007,-0.89528,86.86034]
              ])
C_M_N4 = np.array([[105.30560,-79.80541,-1.58229,-23.78888,-0.09290]
                   ,[-79.80541,326.48530,-62.62674,-57.28538,-70.37290]
                   ,[-1.58229,-62.62674,100.41100,-26.05792,-8.15522]
                   ,[-23.78888,-57.28538,-26.05792,110.20110,-0.72336]
                   ,[-0.09290,-70.37290,-8.15522,-0.72336,79.40180]
                  ])

C_M_N5 = np.array([[119.07130,-84.60930,-2.05832,-32.15181,-0.11997]
                  ,[-84.60930,326.59040,-62.31408,-54.56391,-70.06300]
                  ,[-2.05832,-62.31408,100.36980,-25.74456,-8.16695]
                  ,[-32.15181,-54.56391,-25.74456,115.51110,-0.70659]
                  ,[-0.11997,-70.06300,-8.16695,-0.70659,79.16525]
                  ])

# index dictionary of conductors in input Maxwell capacitance matrix
conductor_index = { 'gnd' : 1
                   , 'island1' : 2
                   , 'island2' : 3
                   , 'bus1' : 0
                   , 'read' : 4
                  }

In [10]:
cap_result = calculate_qubit_1bus_1read_equiv_cap(C_M, conductor_index)
print_result(cap_result)

C_bus1 = -16.292 fF
C_read = 4.849 fF
C_shunt = 61.582 fF


In [11]:
cap_result_N4 = calculate_qubit_1bus_1read_equiv_cap(C_M_N4, conductor_index)
print_result(cap_result_N4)

C_bus1 = -10.320 fF
C_read = 3.990 fF
C_shunt = 71.860 fF


In [12]:
cap_result_N5 = calculate_qubit_1bus_1read_equiv_cap(C_M_N5, conductor_index)
print_result(cap_result_N5)

C_bus1 = -13.471 fF
C_read = 4.139 fF
C_shunt = 75.826 fF


In [13]:
import transmonCPW
from scipy.constants import h,e,k

In [14]:
Ec = e**2/2/75.82e-15/h/1e9
q = transmonCPW.Transmon.in_GHz(20, Ec)

In [15]:
Ec

0.2554765144691573

In [16]:
q.anharmonicity/1e6

-283.83999835346816

In [17]:
q.f01/1e6

6126.509351275784

In [18]:
q.Ej_to_Ec_ratio

78.28508245290986

In [19]:
q.charge_dispersion

2361.203378677368

In [20]:
a= np.array([0,1])

In [22]:
a.

['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'alignment',
 'base',
 'byteorder',
 'char',
 'descr',
 'fields',
 'flags',
 'hasobject',
 'isalignedstruct',
 'isbuiltin',
 'isnative',
 'itemsize',
 'kind',
 'metadata',
 'name',
 'names',
 'ndim',
 'newbyteorder',
 'num',
 'shape',
 'str',
 'subdtype',
 'type']